# BERT 多类微调（Colab 快速启动）

本笔记本用于在 Google Colab 上使用 GPU 训练多类标签模型。

步骤概览：
1. 安装依赖并克隆仓库
2. 上传三份多类数据集 CSV 到 `data/processed/`
3. 运行训练脚本（使用 `label_multi_cls` 标签列）
4. 查看指标并打包下载模型输出

提示：Runtime → Change runtime type → Hardware accelerator 选择 GPU。

In [ ]:
# 安装依赖（transformers/datasets/accelerate/evaluate/scikit-learn）
!pip -q install -U transformers datasets accelerate evaluate scikit-learn

In [ ]:
# 克隆仓库并进入目录
!git clone https://github.com/Caria-Tarnished/Graduation_Project.git -b main
%cd Graduation_Project

In [ ]:
# 上传三份多类训练集到 data/processed/
import os
os.makedirs('data/processed', exist_ok=True)
from google.colab import files
for name in ['train_multi_labeled.csv','val_multi_labeled.csv','test_multi_labeled.csv']:
    print('请上传:', name)
    up = files.upload()  # 在弹窗中选择对应文件
    fname = list(up.keys())[0]
    os.replace(fname, f'data/processed/{name}')
print('上传完成')

In [ ]:
# 启动训练（多类标签：label_multi_cls）
!python scripts/modeling/bert_finetune_cls.py \
  --train_csv data/processed/train_multi_labeled.csv \
  --val_csv   data/processed/val_multi_labeled.csv \
  --test_csv  data/processed/test_multi_labeled.csv \
  --output_dir models/bert_xauusd_multilabel \
  --label_col label_multi_cls \
  --model_name hfl/chinese-roberta-wwm-ext \
  --train_bs 16 --eval_bs 32 --epochs 2 --lr 2e-5 --max_length 256

In [ ]:
# 查看指标（val/test）
import json, os
for nm in ['metrics_val.json','metrics_test.json']:
    p = os.path.join('models/bert_xauusd_multilabel', nm)
    if os.path.exists(p):
        with open(p, 'r', encoding='utf-8') as f:
            print(nm, json.load(f))
    else:
        print('missing:', p)

In [ ]:
# 打包并下载输出目录
import shutil, os
from google.colab import files
out_dir = 'models/bert_xauusd_multilabel'
zip_path = 'bert_xauusd_multilabel.zip'
if os.path.isdir(out_dir):
    if os.path.exists(zip_path):
        os.remove(zip_path)
    shutil.make_archive('bert_xauusd_multilabel', 'zip', out_dir)
    files.download(zip_path)
else:
    print('输出目录不存在：', out_dir)